In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('train.tsv',sep='\t',header=None)
data.columns = ['words','pos']
data

,words,pos
0,ansin,N
1,),N
2,tá,N
3,níos,N
4,lú,N
...,...,...
5057054,do,N
5057055,daoine,S
5057056,beaga,N
5057057,.,N


In [3]:
data=data.astype(str)
data

,words,pos
0,ansin,N
1,),N
2,tá,N
3,níos,N
4,lú,N
...,...,...
5057054,do,N
5057055,daoine,S
5057056,beaga,N
5057057,.,N


In [4]:
lengths = data["words"].str.len()
argmax = np.where(lengths == lengths.max())[0]
data.iloc[argmax]

,words,pos
2286166,\tN\nní\tN\ndóigh\tN\nliom\tN\ngo\tN\nfuil\tU\...,N


In [5]:
len(data['words'][2286166])

57291

In [6]:
def bigram(df):
  gram = []
  for i in range(0,len(df['words'])-1):
    gram.append(df['words'][i]+' '+df['words'][i+1])
  return gram


In [7]:
k = bigram(data)

In [8]:
l = data['pos'][1:].tolist()

In [9]:
k

['ansin )',
 ') tá',
 'tá níos',
 'níos lú',
 'lú gaeilge',
 'gaeilge ag',
 'ag na',
 'na gardaí',
 'gardaí ná',
 'ná bí',
 'bí ariamh',
 'ariamh ainneoin',
 'ainneoin na',
 'na cearta',
 'cearta .',
 '. níl',
 'níl sé',
 'sé ach',
 'ach roinnt',
 'roinnt seachtainí',
 'seachtainí ó',
 'ó sin',
 'sin a',
 'a tógadh',
 'tógadh fear',
 'fear bocht',
 'bocht a',
 'a tug',
 'tug ainm',
 'ainm gaeilge',
 'gaeilge dóibh',
 'dóibh .',
 '. <S>',
 '<S> socraíodh',
 'socraíodh go',
 'go raibh',
 'raibh gá',
 'gá lena',
 'lena leithéid',
 'leithéid ,',
 ', mar',
 'mar go',
 'go bíonn',
 'bíonn na',
 'na <S>',
 '<S> tá',
 'tá an',
 'an córas',
 'córas bainistíochta',
 'bainistíochta tar',
 'tar éis',
 'éis freastal',
 'freastal go',
 'go maith',
 'maith ar',
 'ar rialtas',
 'rialtas áitiúil',
 'áitiúil na',
 'na éireann',
 'éireann agus',
 'agus leanfaidh',
 'leanfaidh údaráis',
 'údaráis áitiúla',
 'áitiúla ar',
 'ar aghaidh',
 'aghaidh ag',
 'ag brath',
 'brath ar',
 'ar bainisteoirí',
 'bainist

In [10]:
# BIGRAM TUPLE LIST



lis=[]
for i in range(len(k)):
   lis.append((k[i],l[i]))
  


In [11]:
lis

[('ansin )', 'N'),
 (') tá', 'N'),
 ('tá níos', 'N'),
 ('níos lú', 'N'),
 ('lú gaeilge', 'N'),
 ('gaeilge ag', 'N'),
 ('ag na', 'N'),
 ('na gardaí', 'N'),
 ('gardaí ná', 'N'),
 ('ná bí', 'S'),
 ('bí ariamh', 'N'),
 ('ariamh ainneoin', 'N'),
 ('ainneoin na', 'N'),
 ('na cearta', 'U'),
 ('cearta .', 'N'),
 ('. níl', 'N'),
 ('níl sé', 'N'),
 ('sé ach', 'N'),
 ('ach roinnt', 'N'),
 ('roinnt seachtainí', 'N'),
 ('seachtainí ó', 'N'),
 ('ó sin', 'S'),
 ('sin a', 'N'),
 ('a tógadh', 'N'),
 ('tógadh fear', 'N'),
 ('fear bocht', 'N'),
 ('bocht a', 'N'),
 ('a tug', 'S'),
 ('tug ainm', 'N'),
 ('ainm gaeilge', 'N'),
 ('gaeilge dóibh', 'N'),
 ('dóibh .', 'N'),
 ('. <S>', 'N'),
 ('<S> socraíodh', 'N'),
 ('socraíodh go', 'N'),
 ('go raibh', 'N'),
 ('raibh gá', 'N'),
 ('gá lena', 'N'),
 ('lena leithéid', 'N'),
 ('leithéid ,', 'N'),
 (', mar', 'N'),
 ('mar go', 'N'),
 ('go bíonn', 'U'),
 ('bíonn na', 'N'),
 ('na <S>', 'N'),
 ('<S> tá', 'N'),
 ('tá an', 'N'),
 ('an córas', 'N'),
 ('córas bainistíochta',

In [12]:
# UNIGRAM TUPLE LIST

m = data['words'].tolist()
n = data['pos'].tolist()
liss=[]
for i in range(len(m)):
   liss.append((m[i],n[i]))
  


In [13]:
test_set = liss[4551353:]
train_set = liss[:4551353] 

In [14]:
  # j = df['pos'][1:].tolist()
  # b_gram = [ x + y for x, y in zip(gram, j)]

In [15]:
# data = data.groupby((np.repeat(np.arange(len(data)), 2)[:len(data)])).agg(' '.join)
# data['pos'] = data['pos'].str[2:]
# data['pos'][2528529] = 'N'

In [16]:
(data['pos']=='N').sum()

4339849

In [17]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_set}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_set}
print(len(vocab))

5
{'U', 'N', 'S', 'H', 'T'}
118706


In [18]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_set):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return (count_w_given_tag, count_tag)

In [19]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_set):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [20]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[6.5690250e-04 9.6020377e-01 3.8334955e-02 7.3733955e-04 6.7030865e-05]
 [3.8031381e-02 8.4304070e-01 1.0574059e-01 9.1096396e-03 4.0774336e-03]
 [1.1575507e-03 9.4543523e-01 5.1788010e-02 1.5584147e-03 6.0805196e-05]
 [9.3112420e-04 9.5785296e-01 3.8696427e-02 2.3825825e-03 1.3693003e-04]
 [1.2524266e-04 9.3869370e-01 5.8112592e-02 2.9432024e-03 1.2524266e-04]]


In [21]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,U,N,S,H,T
U,0.000657,0.960204,0.038335,0.000737,0.000067
N,0.038031,0.843041,0.105741,0.009110,0.004077
S,0.001158,0.945435,0.051788,0.001558,0.000061
H,0.000931,0.957853,0.038696,0.002383,0.000137
T,0.000125,0.938694,0.058113,0.002943,0.000125


In [25]:
def Viterbi(words, train_bag = train_set):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['N', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [26]:
# import random
# # Let's test our Viterbi algorithm on a few sample sentences of test dataset
# random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# # choose random 10 numbers
# rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# # list of 10 sents on which we test the model
# test_run = [test_set[i] for i in rndom]
 
# # list of tagged words
# test_run_base = [tup for sent in test_run for tup in sent]
 
# # list of untagged words
# test_set = [tup[0] for sent in test_run for tup in sent]a

In [33]:
test_set = train_set[4551343:]
test_set
test_run_base = [tup for sent in test_set for tup in sent]

['dá',
 'N',
 'contae',
 'S',
 'sa',
 'N',
 'dáilcheantar',
 'N',
 ')',
 'N',
 '.',
 'N',
 '<S>',
 'N',
 'bí',
 'S',
 'ciall',
 'N',
 'lena',
 'N']

In [34]:
import time
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_set)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  44.1335723400116
Viterbi Algorithm Accuracy:  0.0
